In [1]:
import requests
from bs4 import BeautifulSoup
from typing import Dict, Any, List
import time

In [2]:
def check_media(html_parse):
    """
    
    """
    media_type = html_parse.find("span", class_="ico--media-type")
    if media_type:
        return None
    
    links = html_parse.find("a")
    if links:
        url = "https://www.voatibetan.com" + links.get("href")
        return url

    


In [57]:

def extract_all_VOATibetan_article_links(url: str) -> Dict[str, Any]:
    """
    Extracts all article links from a given VOATibetan webpage.

    Args:
    url (str): The URL of the VOATibetan webpage containing article links.

    Returns:
    Dict[str, Any]: A dictionary containing article links and status details.
    """
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    final_response = {
        "Links": [],
        "Message": "Success",
        "Response": 200,
        "source_url": url
    }
    load_more = False
    
    try:
        start_time = time.time()
        response = requests.get(url, headers=headers, timeout=(5, 60-5))
        response.raise_for_status()
        end_time = time.time()
        if end_time - start_time > 50:
            print(f"This URL took more than 50s: {url}")
        soup = BeautifulSoup(response.content, 'html.parser')
        
        article_div = soup.find("div", class_="col-xs-12 col-md-8 col-lg-8 pull-left content-offset")
        if not article_div:
            raise ValueError("Could not find the main article container on the page.")
        first_link = article_div.find("div", class_="media-block")

        
        article_links = []
        link1 = check_media(first_link)
        article_links.append(link1)
        
        all_articles_div = article_div.find("ul", id="ordinaryItems")
        if not all_articles_div:
            raise ValueError("Could not find the each article container on the page.")
        
        all_articles = all_articles_div.find_all("div", class_="media-block")
        for article in all_articles:
            link = check_media(article)
            if link:
                article_links.append(link)

        final_response["Links"] = article_links

        load_more_span = soup.find("p", class_="buttons btn--load-more")
        if load_more_span:
            load_more = True

        return final_response, load_more
    
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408
        return final_response, True
    except requests.RequestException as e:
        final_response["Message"] = f"An error occurred while fetching the webpage: {e}"
        final_response["Response"] = getattr(e.response, 'status_code', 500)
        return final_response, True
    except ValueError as e:
        final_response["Message"] = f"An error occurred while parsing the webpage: {e}"
        final_response["Response"] = getattr(e.response, 'status_code', 500)
        return final_response, True
    except Exception as e:
        final_response["Message"] = f"An unexpected error occurred: {e}"
        final_response["Response"] = 500
        return final_response, True



In [58]:
url = "https://www.voatibetan.com/z/2253?p=50"

extract_all_VOATibetan_article_links(url)

({'Links': ['https://www.voatibetan.com/a/china-begins-campaign-to-monitor-anti-chinese-propagandas-in-tar/7345642.html',
   'https://www.voatibetan.com/a/authorities-arrest-a-tibetan-monk-in-zokhe-in-eastern-tibet/7345458.html',
   'https://www.voatibetan.com/a/his-holiness-the-dalai-lama-gave-audience-to-the-tibetan-settlement-officers/7345397.html',
   'https://www.voatibetan.com/a/gold-medal-from-mangaluru-university-of-karnatka-state/7344069.html',
   'https://www.voatibetan.com/a/the-dalai-lama-sends-condolence-message-to-the-prime-minister-of-nepal/7343685.html'],
  'Message': 'Success',
  'Response': 200,
  'source_url': 'https://www.voatibetan.com/z/2253?p=50'},
 True)

In [59]:
url = "https://www.voatibetan.com/z/2252?p=2"
extract_all_VOATibetan_article_links(url)

({'Links': ['https://www.voatibetan.com/a/speaker-emerita-pelosi-warns-in-her-new-book-that-political-threats-violence-must-stop-/7733014.html',
   'https://www.voatibetan.com/a/harris-selects-tim-walz-as-running-mate-for-us-presidential-run/7732016.html',
   'https://www.voatibetan.com/a/trump-questions-harris-race-at-gathering-of-black-journalists-/7727171.html',
   'https://www.voatibetan.com/a/blinken-hails-closer-ties-with-mongolia-in-rare-visit-/7727156.html'],
  'Message': 'Success',
  'Response': 200,
  'source_url': 'https://www.voatibetan.com/z/2252?p=2'},
 True)

In [69]:

def scrape_VOATibetan_article_content(url, Tags=""):
    """
    
    
    """


    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    final_response = {
        "data": {
            'title': "",
            'body': {"Audio": "", "Text": []},
            'meta_data': {'URL': url, 'Author': "", 'Date': "", 'Tags': [Tags]}
        },
        "Message": "Success",
        "Response": 200
    }
    
    try:
        # Make the request to the URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # getting title, date, tags
        # section role="header"
        title_data = soup.find("div", class_="col-title col-xs-12 col-md-10 pull-right")        
        # Extract title
        final_response['data']["title"] = title_data.get_text(strip=True) if title_data else ""

        # publishing-details and date
        date_author = soup.find("div", class_="publishing-details")
        if date_author:
            author_div = date_author.find("span", class_="date")
            author = author_div.get_text(strip=True) if author_div else ""

            date_div = date_author.find("a", class_="links__item-link")
            date_time = date_div.get_text(strip=True) if date_div else ""
        else:
            author_div = ""
            date_time = None
        final_response['data']['meta_data']["Author"] = author
        final_response['data']['meta_data']["Date"] = date_time

        
        # Extract audio content
        try:
            # Find the main audio link  wsw__embed
            audio_div = soup.find_all('div', class_='c-mmp__player')
            if len(audio_div):
                audio_sources = []
                # print(audio_div)
                for audio_div in audio_div:
                    # Find the audio element within each div
                    audio = audio_div.find('audio')
                    if audio and audio.get("src"):
                        audio_sources.append(audio.get("src"))
                final_response['data']['body']["Audio"] = audio_sources
            else:
                final_response['data']['body']["Audio"] = ""
        except AttributeError as e:
            final_response['data']['body']["Audio"] = ""
        
        # Extract body content
        try:
            # Find the main content div
            content_div = soup.find('div', class_='wsw')
            if content_div:
                # excluding wsw__embed elements which has all audio
                for embed in content_div.find_all(class_='wsw__embed'):
                    embed.decompose()
                # Extract all text content, 
                main_content = []
                # Get remaining text, excluding empty lines
                remaining_text = content_div.get_text(strip=True, separator='\n').split('\n')
                main_content.extend([text for text in remaining_text if text])
                
                final_response['data']['body']["Text"] = main_content
            else:
                final_response['data']['body']["Text"] = ["No Content in the article"]
        except AttributeError as e:
            final_response['data']['body']["Text"] = [f"Error fetching body content{str(e)}"]

        
        return final_response
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408  # Request Timeout
        return final_response
        
    except requests.RequestException as e:
        final_response["Message"] = f"An error occurred while fetching the article: {str(e)}"
        final_response["Response"] = getattr(e.response, 'status_code', 500)
        return final_response
    
    except Exception as e:
        final_response["Message"] = f"An error occurred in code: {str(e)}"
        final_response["Response"] = 404
        return final_response





In [70]:
url = "https://www.voatibetan.com/a/harris-selects-tim-walz-as-running-mate-for-us-presidential-run/7732016.html"
scrape_VOATibetan_article_content(url)

{'data': {'title': 'ཊིམ་ཝོལ་ཛི་དམངས་གཙོ་ཚོགས་པའི་སྲིད་འཛིན་གཞོན་པའི་འོས་མིར་བདམས།',
  'body': {'Audio': '',
   'Text': ['༄༅།། སྲིད་འཛིན་གཞོན་པ་ཀ་མ་ལ་ཧེ་རི་སིས་དམངས་གཙོ་ཚོགས་པའི་སྲིད་འཛིན་གཞོན་པའི་འོས་མི་དེར་མཱི་ནི་སོ་ཊ་མངའ་སྡེའི་སྤྱི་ཁྱབ་ཊིམ་ཝོལ་ཛི་(Tim Walz)་བདམས་ཡོད་ཅིང་། ད་ལོའི་ཕྱི་ཟླ་༡༡་པའི་སྲིད་འཛིན་འོས་བསྡུའི་སྐབས་སུ་སྤྱི་མཐུན་ཚོགས་པའི་སྲིད་འཛིན་འོས་མི་ཌོན་ནལ་ཊམ་དང་སྲིད་འཛིན་གཞོན་པའི་འོས་མི་ཇཱེ་ཌི་ཝན་སི་གཉིས་ལ་འགྲན་སྡུར་བྱས་ནས་འོས་བསྡུའི་དྲིལ་བསྒྲགས་ལས་འགུལ་སྤེལ་རྒྱུའི་ཐག་བཅད།།',
    'དེ་རིང་གཟའ་མིག་དམར་གྱི་སྔ་དྲོར་ཧྥི་ལ་ཌལ་ཧྥི་ཡ་གྲོང་ཁྱེར་དུ་དམངས་གཙོ་ཚོགས་པའི་སྲིད་འཛིན་འོས་མི་ཀ་མ་ལ་ཧེ་རི་སིས་འོས་བསྡུའི་དྲིལ་བསྒྲགས་ལས་འགུལ་སྤེལ་བཞིན་པའི་སྐབས་སུ་སྲིད་འཛིན་གཞོན་པའི་འོས་མིའི་མིང་ཐོ་ཕྱིར་བསྒྲགས་བྱས་འདུག ཁོང་གིས་ངས་མཱི་ནི་སོ་ཊ་མངའ་སྡེའི་སྤྱི་ཁྱབ་ཊིམ་ཝོལ་ཛི་སྲིད་འཛིན་གཞོན་པའི་འོས་མིར་གདམ་རྒྱུའི་ཐག་གཅོད་བྱས་པ་ཡིན། ཊིམ་ནི་འཐབ་རྩོད་ཀྱི་ཉམས་མྱོང་ལྡན་པའི་དབུ་ཁྲིད་ཅིག་དང་མཱི་ནི་སོ་ཊ་མངའ་སྡེའི་ཁྱིམ་ཚང་ཁག་གི་མང་ཚོགས་ཀྱི་བདེ་སྡུག་ལ་གཟིགས་རྟོག་གནང་བའི་མཛད་རྗེས་གཟེངས་སུ་ཐོན་པ་སྟོན་རྒྱུ་ཡོད་པའི་མི

In [71]:
url = "https://www.voatibetan.com/a/his-holiness-the-dalai-lama-gave-audience-to-the-tibetan-settlement-officers/7345397.html"
scrape_VOATibetan_article_content(url)

{'data': {'title': '༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ།',
  'body': {'Audio': ['https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb.mp3'],
   'Text': ['བོད་ཀྱི་བླ་ན་མེད་པའི་དབུ་ཁྲིད་སྤྱི་ནོར་༧གོང་ས་༧སྐྱབས་མགོན་ཆེན་པོ་མཆོག་ལ་བོད་མིའི་སྒྲིག་འཛུགས་ནང་སྲིད་ལས་ཁུངས་ཀྱི་ཁྱབ་ཁོངས་ས་གནས་འགོ་འཛིན་གྱི་འཛིན་སྐྱོང་ཚོགས་ཆེན་ཐེངས་ ༡༥ པའི་གོ་སྒྲིག་ཚོགས་ཆུང་གིས་ཇི་ལྟར་གསོལ་འདེབས་ཞུས་པ་བཞིན་ཕྱི་ཟླ་ ༡༡ ཚེས་ ༧ ཉིན་མགོན་པོ་མཆོག་ནས་བཞུགས་སྒར་རྒྱལ་བའི་ཕོ་བྲང་དུ་ཁོང་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་ཡོད་པ་རེད།',
    'མཇལ་ཁའི་སྐབས་༧གོང་ས་༧སྐྱབས་མགོན་ཆེན་པོ་མཆོག་ནས་སྩལ་བའི་གནང་བའི་བཀའ་སློབ་ནང་། རྒྱུན་དུ་ཁོང་ཁྲོ་དང་ང་རྒྱལ་སོགས་ཀྱི་དབང་གིས་མི་དང་མིའི་བར་ལ་མི་འཆམ་པའམ་རྩོད་པའི་གཞི་ཆགས་པའི་ཚེ་མགོན་པོ་གང་ཉིད་ཀྱི་ཞལ་རས་ཡིད་ལ་དྲན་ཏེ་སེམས་བག་ལྷོད་དུ་གཏོང་དགོས་པའི་བཀའ་སློབ་དང་སྦྲགས་རྒྱུན་དུ་༧གོང་ས་མཆོག་ནས་བོད་ལ་མིང་དོན་མཚུངས་པའི་རང་སྐྱོང་གཙང་མ་ཞིག་དགོས་ཀྱི་འདུག་ཅེས་གསུང་གི་ཡོད་པ་ལྟར། རྒྱ་ནག་མི་མང་སྤྱི་མཐུན་རྒྱལ་ཁབ་ཀྱི་རྩ་ཁྲིམས་འོག་ཏུ་བོད་ཀྱི་ཆོས་དང

In [18]:
from bs4 import BeautifulSoup

html = """
<<div class="col-xs-12 col-sm-12 col-md-8 col-lg-8 pull-left bottom-offset content-offset">
<div id="article-content" class="content-floated-wrap fb-quotable">
<div class="wsw">
<div>བོད་ཀྱི་བླ་ན་མེད་པའི་དབུ་ཁྲིད་སྤྱི་ནོར་༧གོང་ས་༧སྐྱབས་མགོན་ཆེན་པོ་མཆོག་ལ་བོད་མིའི་སྒྲིག་འཛུགས་ནང་སྲིད་ལས་ཁུངས་ཀྱི་ཁྱབ་ཁོངས་ས་གནས་འགོ་འཛིན་གྱི་འཛིན་སྐྱོང་ཚོགས་ཆེན་ཐེངས་ ༡༥ པའི་གོ་སྒྲིག་ཚོགས་ཆུང་གིས་ཇི་ལྟར་གསོལ་འདེབས་ཞུས་པ་བཞིན་ཕྱི་ཟླ་ ༡༡ ཚེས་ ༧ ཉིན་མགོན་པོ་མཆོག་ནས་བཞུགས་སྒར་རྒྱལ་བའི་ཕོ་བྲང་དུ་ཁོང་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་ཡོད་པ་རེད།</div>
<div> </div>
<div>མཇལ་ཁའི་སྐབས་༧གོང་ས་༧སྐྱབས་མགོན་ཆེན་པོ་མཆོག་ནས་སྩལ་བའི་གནང་བའི་བཀའ་སློབ་ནང་། རྒྱུན་དུ་ཁོང་ཁྲོ་དང་ང་རྒྱལ་སོགས་ཀྱི་དབང་གིས་མི་དང་མིའི་བར་ལ་མི་འཆམ་པའམ་རྩོད་པའི་གཞི་ཆགས་པའི་ཚེ་མགོན་པོ་གང་ཉིད་ཀྱི་ཞལ་རས་ཡིད་ལ་དྲན་ཏེ་སེམས་བག་ལྷོད་དུ་གཏོང་དགོས་པའི་བཀའ་སློབ་དང་སྦྲགས་རྒྱུན་དུ་༧གོང་ས་མཆོག་ནས་བོད་ལ་མིང་དོན་མཚུངས་པའི་རང་སྐྱོང་གཙང་མ་ཞིག་དགོས་ཀྱི་འདུག་ཅེས་གསུང་གི་ཡོད་པ་ལྟར། རྒྱ་ནག་མི་མང་སྤྱི་མཐུན་རྒྱལ་ཁབ་ཀྱི་རྩ་ཁྲིམས་འོག་ཏུ་བོད་ཀྱི་ཆོས་དང་རིག་གཞུང་སོགས་བོད་པ་རང་གིས་བདག་ཉར་བྱེད་ཐུབ་པ་ཞིག་དང་། དེས་རྒྱ་མི་རིགས་ལ་ཡང་ཕན་ཐོགས་བྱེད་ཐུབ་པ་ཞིག་ཡོང་གི་ཡོད་པའི་སྐོར་ལའང་བཀའ་སློབ་སྩལ་གནང་འདུག་ལ། འཛམ་བུ་གླིང་འདིའི་ནང་སྐྱབས་བཅོལ་བ་མང་པོ་ཡོད་པའི་ནང་ནས་བོད་མི་སྐྱབས་བཅོལ་བ་ཚོས་སོ་སོའི་ཆོས་དང་རིག་གཞུང་མུ་མཐུད་མི་ཉམས་རྒྱུན་འཛིན་གནང་མཁན་ཞིག་ཡིན་པ་མ་ཟད། བོད་ཀྱི་ཆོས་དང་རིག་གཞུང་གི་ཐོག་ནས་མི་ལྷོད་ལྷོད་ཅིག་ཆགས་ཀྱི་ཡོད་པ་སོགས་ལའང་བཀའ་སློབ་སྩལ་གནང་འདུག</div>
<div> </div>
<div> </div>
<div><div class="wsw__embed">
<figure class="media-image js-media-expand">
<div class="img-wrap">
<div class="thumb">
<img alt="" src="https://gdb.voanews.com/01000000-c0a8-0242-b810-08dbdfbf0f19_w250_r1_s.jpg" />
</div>
<span class="ico ico-fullscreen ico--media-expand ico--rounded" />
</div>
</figure>
</div>
གཞི་རྩའི་ད་རེས་ཀྱི་མཇལ་བཅར་བ་རྣམས་ནི། འདི་ཚེས་ ༦ ཉིན་འགོ་འཛུགས་གནང་བའི་ས་གནས་འགོ་འཛིན་རྣམས་ཀྱི་འཛིན་སྐྱོང་ཚོགས་ཆེན་ཐེངས་ ༡༥ ནང་མཉམ་བཞུགས་གནང་མཁན་རྣམས་ཡིན་པ་དང་། ད་རེས་ཀྱི་ཚོགས་ཆེན་འདིའི་ཐོག་བལ་ཡུལ་གྱི་ས་གནས་གཅིག་ཕུད་པའི་གཞན་ས་གནས་ཁག་ཚང་མའི་ས་གནས་འགོ་འཛིན་ ༤༥ ལྷན་འཛོམས་ཀྱིས། དེ་སྔ་འཚོགས་གནང་བའི་སྐབས་ ༡༤ པའི་འཛིན་སྐྱོང་ཚོགས་འདུའི་སྐབས་ཀྱི་གྲོས་ཆོད་ཁག་ལ་དབུས་དང་ས་གནས་གཉིས་ནས་ལག་བསྟར་ཇི་ལྟར་གནང་ཡོད་མེད་ཀྱི་སྙན་སྒྲོན་དང་སྦྲགས། གཙོ་བོ་གྲོས་གཞི་དོན་ཚན་བརྒྱད་ཀྱི་ཐད་ལ་བཀའ་བསྡུར་དང་། དེ་དག་ཀྱང་གཙོ་བོ་ས་གནས་འགོ་འཛིན་རྣམ་པའི་འཛིན་སྐྱོང་གི་ལས་དོན་ལ་འགན་ཁུར་ཞུ་ཕྱོགས་གཙོས་པས་བཙན་བྱོལ་བོད་མིའི་གཞིས་ཆགས་ཁག་གི་ས་ཁང་དང་འབྲེལ་བའི་སྐོར། དེ་བཞན་གཞིས་ཆགས་ཁག་ལ་ས་ཞིང་འདེ་བས་ལས་བྱེད་ཕྱོགས། ས་གནས་ཁག་ལ་ལས་གཞི་གསར་སྐྲུན། ན་གཞོན་རོགས་སྐྱོར་སོགས་ཀྱི་སྐོར་ལ་ཡིན་པ་ནང་སྲིད་དྲུང་ཆེ་དཔལ་ལྡན་དོན་གྲུབ་ལགས་ཀྱིས་འགྲེལ་བརྗོད་གནང་བྱུང་།
<div class="wsw__embed">
<div class="media-pholder media-pholder--audio media-pholder--embed">
<div class="c-sticky-container" data-poster="https://gdb.voanews.com/01000000-c0a8-0242-b810-08dbdfbf0f19_w250_r1.jpg">
<div class="c-sticky-element" data-sp_api="pangea-video" data-persistent data-persistent-browse-out >
<div class="c-mmp c-mmp--enabled c-mmp--loading c-mmp--audio c-mmp--embed c-mmp--has-poster c-sticky-element__swipe-el"
data-player_id="" data-title="༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། " data-hide-title="False"
data-breakpoint_s="320" data-breakpoint_m="640" data-breakpoint_l="992"
data-hlsjs-src="/Scripts/responsive/hls.b"
data-bypass-dash-for-vod="true"
data-bypass-dash-for-live-video="true"
data-bypass-dash-for-live-audio="true">
<div class="c-mmp__poster js-poster">
<img src="https://gdb.voanews.com/01000000-c0a8-0242-b810-08dbdfbf0f19_w250_r1.jpg" alt="༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། " title="༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། " />
</div>
<a class="c-mmp__fallback-link" href="https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb.mp3">
<span class="c-mmp__fallback-link-icon">
<span class="ico ico-audio"></span>
</span>
</a>
<div class="c-spinner">
<img src="/Content/responsive/img/player-spinner.png" alt="please wait" title="please wait" />
</div>
<div class="c-mmp__player">
<audio src="https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb.mp3" data-fallbacksrc="" data-fallbacktype="" data-type="audio/mp3" data-info="64 kbps" data-sources="[{&quot;AmpSrc&quot;:&quot;https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb_hq.mp3&quot;,&quot;Src&quot;:&quot;https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb_hq.mp3&quot;,&quot;Type&quot;:&quot;audio/mp3&quot;,&quot;DataInfo&quot;:&quot;128 kbps&quot;,&quot;Url&quot;:null,&quot;BlockAutoTo&quot;:null,&quot;BlockAutoFrom&quot;:null},{&quot;AmpSrc&quot;:&quot;https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb_32k.mp3&quot;,&quot;Src&quot;:&quot;https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb_32k.mp3&quot;,&quot;Type&quot;:&quot;audio/mp3&quot;,&quot;DataInfo&quot;:&quot;32 kbps&quot;,&quot;Url&quot;:null,&quot;BlockAutoTo&quot;:null,&quot;BlockAutoFrom&quot;:null}]" data-lt-on-play="0" data-lt-url="" webkit-playsinline="webkit-playsinline" playsinline="playsinline" style="width:100%;height:140px" title="༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། ">
</audio>
</div>
<div class="c-mmp__overlay c-mmp__overlay--title c-mmp__overlay--partial c-mmp__overlay--disabled c-mmp__overlay--slide-from-top js-c-mmp__title-overlay">
<span class="c-mmp__overlay-actions c-mmp__overlay-actions-top js-overlay-actions">
<span class="c-mmp__overlay-actions-link c-mmp__overlay-actions-link--embed js-btn-embed-overlay" title="Embed">
<span class="c-mmp__overlay-actions-link-ico ico ico-embed-code"></span>
<span class="c-mmp__overlay-actions-link-text">Embed</span>
</span>
<span class="c-mmp__overlay-actions-link c-mmp__overlay-actions-link--share js-btn-sharing-overlay" title="share">
<span class="c-mmp__overlay-actions-link-ico ico ico-share"></span>
<span class="c-mmp__overlay-actions-link-text">share</span>
</span>
<span class="c-mmp__overlay-actions-link c-mmp__overlay-actions-link--close-sticky c-sticky-element__close-el" title="ཁ་རྒྱག་པ།">
<span class="c-mmp__overlay-actions-link-ico ico ico-close"></span>
</span>
</span>
<div class="c-mmp__overlay-title js-overlay-title">
<h5 class="c-mmp__overlay-media-title">
<a class="js-media-title-link" href="/a/7345377.html" target="_blank" rel="noopener" title="༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། ">༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། </a>
</h5>
<div class="c-mmp__overlay-site-title">
<small>by <a href="https://www.voatibetan.com" target="_blank" rel="noopener" title="ཨ་རིའི་རླུང་འཕྲིན་ཁང་།">ཨ་རིའི་རླུང་འཕྲིན་ཁང་།</a></small>
</div>
</div>
</div>
<div class="c-mmp__overlay c-mmp__overlay--sharing c-mmp__overlay--disabled c-mmp__overlay--slide-from-bottom js-c-mmp__sharing-overlay">
<span class="c-mmp__overlay-actions">
<span class="c-mmp__overlay-actions-link c-mmp__overlay-actions-link--embed js-btn-embed-overlay" title="Embed">
<span class="c-mmp__overlay-actions-link-ico ico ico-embed-code"></span>
<span class="c-mmp__overlay-actions-link-text">Embed</span>
</span>
<span class="c-mmp__overlay-actions-link c-mmp__overlay-actions-link--share js-btn-sharing-overlay" title="share">
<span class="c-mmp__overlay-actions-link-ico ico ico-share"></span>
<span class="c-mmp__overlay-actions-link-text">share</span>
</span>
<span class="c-mmp__overlay-actions-link c-mmp__overlay-actions-link--close js-btn-close-overlay" title="ཁ་རྒྱག་པ།">
<span class="c-mmp__overlay-actions-link-ico ico ico-close"></span>
</span>
</span>
<div class="c-mmp__overlay-tabs">
<div class="c-mmp__overlay-tab c-mmp__overlay-tab--disabled c-mmp__overlay-tab--slide-backward js-tab-embed-overlay" data-trigger="js-btn-embed-overlay" data-embed-source="//www.voatibetan.com/embed/player/0/7345377.html?type=audio" role="form">
<div class="c-mmp__overlay-body c-mmp__overlay-body--centered-vertical">
<div class="column">
<div class="c-mmp__status-msg ta-c js-message-embed-code-copied" role="tooltip">
The code has been copied to your clipboard.
</div>
<div class="c-mmp__form-group ta-c">
<input type="text" name="embed_code" class="c-mmp__input-text js-embed-code" dir="ltr" value="" readonly />
<span class="c-mmp__input-btn js-btn-copy-embed-code" title="Copy to clipboard"><span class="ico ico-content-copy"></span></span>
</div>
</div>
</div>
</div>
<div class="c-mmp__overlay-tab c-mmp__overlay-tab--disabled c-mmp__overlay-tab--slide-forward js-tab-sharing-overlay" data-trigger="js-btn-sharing-overlay" role="form">
<div class="c-mmp__overlay-body c-mmp__overlay-body--centered-vertical">
<div class="column">
<div class="c-mmp__status-msg ta-c js-message-share-url-copied" role="tooltip">
The URL has been copied to your clipboard
</div>
<div class="not-apply-to-sticky audio-fl-bwd">
<aside class="player-content-share share share--mmp">
<ul class="share__list">
<li class="share__item">
<a href="https://facebook.com/sharer.php?u=https%3a%2f%2fwww.voatibetan.com%2fa%2f7345377.html"
data-analytics-text="share_on_facebook"
title="ངོ་དེབ་ཐོག་ལ་འགྲེམ་སྤེལ།" target="_blank"
class="btn bg-transparent btn-facebook js-social-btn">
<span class="ico fs-xl ico-facebook "></span>
<span class="btn__text sr-only">ངོ་དེབ་ཐོག་ལ་འགྲེམ་སྤེལ།</span>
</a>
</li>
<li class="share__item">
<a href="https://twitter.com/share?url=https%3a%2f%2fwww.voatibetan.com%2fa%2f7345377.html&amp;text=%e0%bc%a7%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%82%e0%bd%b2%e0%bd%a6%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%82%e0%bd%93%e0%bd%a6%e0%bc%8b%e0%bd%a0%e0%bd%82%e0%bd%bc%e0%bc%8b%e0%bd%a0%e0%bd%9b%e0%bd%b2%e0%bd%93%e0%bc%8b%e0%bd%a2%e0%be%a3%e0%bd%98%e0%bd%a6%e0%bc%8b%e0%bd%a3%e0%bc%8b%e0%bd%98%e0%bd%87%e0%bd%a3%e0%bc%8b%e0%bd%81%e0%bc%8b%e0%bd%91%e0%bd%84%e0%bc%8b%e0%bd%96%e0%bd%80%e0%bd%a0%e0%bc%8b%e0%bd%a6%e0%be%b3%e0%bd%bc%e0%bd%96%e0%bc%8b%e0%bd%a6%e0%be%a9%e0%bd%a3%e0%bc%8b%e0%bd%82%e0%bd%93%e0%bd%84%e0%bc%8b%e0%bd%98%e0%bd%9b%e0%bd%91%e0%bc%8b%e0%bd%94%e0%bc%8d+"
data-analytics-text="share_on_twitter"
title="ཌི་ཝི་ཌར་ཐོག་ལ་འགྲེམ་སྤེལ།" target="_blank"
class="btn bg-transparent btn-twitter js-social-btn">
<span class="ico fs-xl ico-twitter "></span>
<span class="btn__text sr-only">ཌི་ཝི་ཌར་ཐོག་ལ་འགྲེམ་སྤེལ།</span>
</a>
</li>
<li class="share__item share__item--sharing-more">
<a title="དེ་ལས་མང་བའི་འགྲེམ་སྤེལ་གྱི་གདམ་ཀ།" data-title-additional="Fewer options" class="btn bg-transparent btn--show-more hidden">
<span class="ico fs-xl ico-ellipsis "></span>
</a>
</li>
<li class="share__item share__item--sharing-more">
<a href="/a/7345377.html" title="Share this media" class="btn bg-transparent" target="_blank" rel="noopener">
<span class="ico ico-ellipsis "></span>
</a>
</li>
</ul>
</aside>
</div>
<hr class="c-mmp__separator-line audio-fl-bwd xs-hidden s-hidden" />
<div class="c-mmp__form-group ta-c audio-fl-bwd xs-hidden s-hidden">
<input type="text" name="share_url" class="c-mmp__input-text js-share-url" value="https://www.voatibetan.com/a/7345377.html" dir="ltr" readonly />
<span class="c-mmp__input-btn js-btn-copy-share-url" title="Copy to clipboard"><span class="ico ico-content-copy"></span></span>
</div>
</div>
</div>
</div>
</div>
</div>
<div class="c-mmp__overlay c-mmp__overlay--settings c-mmp__overlay--disabled c-mmp__overlay--slide-from-bottom js-c-mmp__settings-overlay">
<span class="c-mmp__overlay-actions">
<span class="c-mmp__overlay-actions-link c-mmp__overlay-actions-link--close js-btn-close-overlay" title="ཁ་རྒྱག་པ།">
<span class="c-mmp__overlay-actions-link-ico ico ico-close"></span>
</span>
</span>
<div class="c-mmp__overlay-body c-mmp__overlay-body--centered-vertical">
<div class="column column--scrolling js-sources"></div>
</div>
</div>
<div class="c-mmp__overlay c-mmp__overlay--disabled js-c-mmp__disabled-overlay">
<div class="c-mmp__overlay-body c-mmp__overlay-body--centered-vertical">
<div class="column">
<p class="ta-c"><span class="ico ico-clock"></span>No media source currently available</p>
</div>
</div>
</div>
<div class="c-mmp__cpanel-container js-cpanel-container">
<div class="c-mmp__cpanel c-mmp__cpanel--hidden">
<div class="c-mmp__cpanel-playback-controls">
<span class="c-mmp__cpanel-btn c-mmp__cpanel-btn--play js-btn-play" title="play">
<span class="ico ico-play m-0"></span>
</span>
<span class="c-mmp__cpanel-btn c-mmp__cpanel-btn--pause js-btn-pause" title="pause">
<span class="ico ico-pause m-0"></span>
</span>
</div>
<div class="c-mmp__cpanel-progress-controls">
<span class="c-mmp__cpanel-progress-controls-current-time js-current-time" dir="ltr">0:00</span>
<span class="c-mmp__cpanel-progress-controls-duration js-duration" dir="ltr">
0:03:54
</span>
<span class="c-mmp__indicator c-mmp__indicator--horizontal" dir="ltr">
<span class="c-mmp__indicator-lines js-progressbar">
<span class="c-mmp__indicator-line c-mmp__indicator-line--range js-playback-range" style="width:100%"></span>
<span class="c-mmp__indicator-line c-mmp__indicator-line--buffered js-playback-buffered" style="width:0%"></span>
<span class="c-mmp__indicator-line c-mmp__indicator-line--tracked js-playback-tracked" style="width:0%"></span>
<span class="c-mmp__indicator-line c-mmp__indicator-line--played js-playback-played" style="width:0%"></span>
<span class="c-mmp__indicator-line c-mmp__indicator-line--live js-playback-live"><span class="strip"></span></span>
<span class="c-mmp__indicator-btn ta-c js-progressbar-btn">
<button class="c-mmp__indicator-btn-pointer" type="button"></button>
</span>
<span class="c-mmp__badge c-mmp__badge--tracked-time c-mmp__badge--hidden js-progressbar-indicator-badge" dir="ltr" style="left:0%">
<span class="c-mmp__badge-text js-progressbar-indicator-badge-text">0:00</span>
</span>
</span>
</span>
</div>
<div class="c-mmp__cpanel-additional-controls">
<span class="c-mmp__cpanel-additional-controls-volume js-volume-controls">
<span class="c-mmp__cpanel-btn c-mmp__cpanel-btn--volume js-btn-volume" title="སྒྲ་ཤུགས།">
<span class="ico ico-volume-unmuted m-0"></span>
</span>
<span class="c-mmp__indicator c-mmp__indicator--vertical js-volume-panel" dir="ltr">
<span class="c-mmp__indicator-lines js-volumebar">
<span class="c-mmp__indicator-line c-mmp__indicator-line--range js-volume-range" style="height:100%"></span>
<span class="c-mmp__indicator-line c-mmp__indicator-line--volume js-volume-level" style="height:0%"></span>
<span class="c-mmp__indicator-slider">
<span class="c-mmp__indicator-btn ta-c c-mmp__indicator-btn--hidden js-volumebar-btn">
<button class="c-mmp__indicator-btn-pointer" type="button"></button>
</span>
</span>
</span>
</span>
</span>
<div class="c-mmp__cpanel-additional-controls-settings js-settings-controls">
<span class="c-mmp__cpanel-btn c-mmp__cpanel-btn--settings-overlay js-btn-settings-overlay" title="source switch">
<span class="ico ico-settings m-0"></span>
</span>
<span class="c-mmp__cpanel-btn c-mmp__cpanel-btn--settings-expand js-btn-settings-expand" title="source switch">
<span class="ico ico-settings m-0"></span>
</span>
<div class="c-mmp__expander c-mmp__expander--sources js-c-mmp__expander--sources">
<div class="c-mmp__expander-content js-sources"></div>
</div>
</div>
</div>
</div>
</div>
</div>
</div>
</div>
<div class="media-download">
<div class="simple-menu">
<span class="handler">
<span class="ico ico-download"></span>
<span class="label">ཐད་ཀར་ཕབ་ལེན་གྱི་དྲ་འབྲེལ།</span>
<span class="ico ico-chevron-down"></span>
</span>
<div class="inner">
<ul class="subitems">
<li class="subitem">
<a href="https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb_hq.mp3?download=1" title="128 kbps | MP3" class="handler"
onclick="FireAnalyticsTagEventOnDownload(this, 'audio', 7345377, '༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། ', null, 'བསྟན་འཛིན་དབང་འདུས།', '2023', '11', '08')">
128 kbps | MP3
</a>
</li>
<li class="subitem">
<a href="https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb_32k.mp3?download=1" title="32 kbps | MP3" class="handler"
onclick="FireAnalyticsTagEventOnDownload(this, 'audio', 7345377, '༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། ', null, 'བསྟན་འཛིན་དབང་འདུས།', '2023', '11', '08')">
32 kbps | MP3
</a>
</li>
<li class="subitem">
<a href="https://voa-audio-ns.akamaized.net/vti/2023/11/07/01000000-0a00-0242-26c2-08dbdfbfc6cb.mp3?download=1" title="64 kbps | MP3" class="handler"
onclick="FireAnalyticsTagEventOnDownload(this, 'audio', 7345377, '༧གོང་ས་མཆོག་གིས་ས་གནས་འགོ་འཛིན་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་པ། ', null, 'བསྟན་འཛིན་དབང་འདུས།', '2023', '11', '08')">
64 kbps | MP3
</a>
</li>
</ul>
</div>
</div>
</div>
<button class="btn btn--link btn-popout-player" data-default-display="block" data-popup-url="/pp/7345377/ppt0.html" title="གསན་གཟིགས་གནང་ས།">
<span class="ico ico-arrow-top-right"></span>
<span class="text">གསན་གཟིགས་གནང་ས།</span>
</button>
</div>
</div>
<p> </p>
</div>
</div>
<ul>
<li>
<div class="c-author c-author--hlight">
<div class="media-block">
<a href="/author/བསྟན་འཛིན་དབང་འདུས།/q_ivv" class="img-wrap img-wrap--xs img-wrap--float img-wrap--t-spac">
<div class="thumb thumb1_1 rounded">
<noscript class="nojs-img">
<img src="https://gdb.voanews.com/01460000-0aff-0242-5496-08da748f91d1_w100_r5.jpg" alt="16x9 Image" class="avatar" />
</noscript>
<img alt="16x9 Image" class="avatar" data-src="https://gdb.voanews.com/01460000-0aff-0242-5496-08da748f91d1_w66_r5.jpg" src="" />
</div>
</a>
<div class="c-author__content">
<h4 class="media-block__title media-block__title--author">
<a href="/author/བསྟན་འཛིན་དབང་འདུས།/q_ivv">བསྟན་འཛིན་དབང་འདུས།</a>
</h4>
<div class="c-author__btns m-t-md">
<a class="btn btn-rss btn--social" href="/api/aq_ivvekbivv" title="མངགས་ལེན།">
<span class="ico ico-rss"></span>
<span class="btn__text">མངགས་ལེན།</span>
</a>
</div>
</div>
</div>
</div>
</li>
</ul>
</div>
</div>
"""

soup = BeautifulSoup(html, 'html.parser')

# Find the main content div
content_div = soup.find('div', class_='wsw')

if content_div:
    for embed in content_div.find_all(class_='wsw__embed'):
            embed.decompose()
    
    # Extract all text content, excluding wsw__embed elements
    main_content = []
    # Get remaining text, excluding empty lines
    remaining_text = content_div.get_text(strip=True, separator='\n').split('\n')
    main_content.extend([text for text in remaining_text if text])

    # Print the extracted content
    for item in main_content:
        print(item  + "\n\n")
else:
    print("Content div not found")

བོད་ཀྱི་བླ་ན་མེད་པའི་དབུ་ཁྲིད་སྤྱི་ནོར་༧གོང་ས་༧སྐྱབས་མགོན་ཆེན་པོ་མཆོག་ལ་བོད་མིའི་སྒྲིག་འཛུགས་ནང་སྲིད་ལས་ཁུངས་ཀྱི་ཁྱབ་ཁོངས་ས་གནས་འགོ་འཛིན་གྱི་འཛིན་སྐྱོང་ཚོགས་ཆེན་ཐེངས་ ༡༥ པའི་གོ་སྒྲིག་ཚོགས་ཆུང་གིས་ཇི་ལྟར་གསོལ་འདེབས་ཞུས་པ་བཞིན་ཕྱི་ཟླ་ ༡༡ ཚེས་ ༧ ཉིན་མགོན་པོ་མཆོག་ནས་བཞུགས་སྒར་རྒྱལ་བའི་ཕོ་བྲང་དུ་ཁོང་རྣམས་ལ་མཇལ་ཁ་དང་བཀའ་སློབ་སྩལ་གནང་མཛད་ཡོད་པ་རེད།


མཇལ་ཁའི་སྐབས་༧གོང་ས་༧སྐྱབས་མགོན་ཆེན་པོ་མཆོག་ནས་སྩལ་བའི་གནང་བའི་བཀའ་སློབ་ནང་། རྒྱུན་དུ་ཁོང་ཁྲོ་དང་ང་རྒྱལ་སོགས་ཀྱི་དབང་གིས་མི་དང་མིའི་བར་ལ་མི་འཆམ་པའམ་རྩོད་པའི་གཞི་ཆགས་པའི་ཚེ་མགོན་པོ་གང་ཉིད་ཀྱི་ཞལ་རས་ཡིད་ལ་དྲན་ཏེ་སེམས་བག་ལྷོད་དུ་གཏོང་དགོས་པའི་བཀའ་སློབ་དང་སྦྲགས་རྒྱུན་དུ་༧གོང་ས་མཆོག་ནས་བོད་ལ་མིང་དོན་མཚུངས་པའི་རང་སྐྱོང་གཙང་མ་ཞིག་དགོས་ཀྱི་འདུག་ཅེས་གསུང་གི་ཡོད་པ་ལྟར། རྒྱ་ནག་མི་མང་སྤྱི་མཐུན་རྒྱལ་ཁབ་ཀྱི་རྩ་ཁྲིམས་འོག་ཏུ་བོད་ཀྱི་ཆོས་དང་རིག་གཞུང་སོགས་བོད་པ་རང་གིས་བདག་ཉར་བྱེད་ཐུབ་པ་ཞིག་དང་། དེས་རྒྱ་མི་རིགས་ལ་ཡང་ཕན་ཐོགས་བྱེད་ཐུབ་པ་ཞིག་ཡོང་གི་ཡོད་པའི་སྐོར་ལའང་བཀའ་སློབ་སྩལ་གནང་འདུག་ལ། འཛམ་བུ་གླིང་འདིའི་ནང་སྐྱབས་བཅོལ་བ་མང་པོ་ཡོད་པའི་ནང་ནས་བོད་མི་སྐྱབས་བཅོལ་བ་ཚོས་ས

In [ ]:
<div class="wsw">
<p>༄༅།། some date here </p>
<p> </p>
<p>same here ་སྤྲིངས།། </p><div class="wsw__embed">
<figure class="media-image js-media-expand">
<div class="img-wrap">
<div class="thumb">
<img alt="མཱི་not this།" src="https://gdb.voanews.com/527a323f-d84b-4a85-974c-e8b249307269_w250_r1_s.jpg">
</img></div>
<span class="ico ico-fullscreen ico--media-expand ico--rounded"></span>
</div>
<figcaption>
<span class="caption">not this ཉིན།</span>
</figcaption>
</figure>
</div>
<p>some data here</p>
<p> </p>
<p>same here take this </p>
<p> </p>
<p>need this data  </p>
</div>